In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

from sklearn import metrics

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# import data
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [ ]:
train_x = train_data.reshape(train_data.shape[0], -1).astype('float32')
test_x = test_data.reshape(test_data.shape[0], -1).astype('float32')

# LightGBM

In [39]:
from lightgbm import LGBMClassifier

In [40]:
lgb_model = LGBMClassifier(objective='multiclass',path_smooth = 0.2)

In [47]:
lgb_model.fit(train_x,train_labels,categorical_feature=[0,3])

C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(objective='multiclass', path_smooth=0.2)

In [107]:
expected_y  = test_labels
predicted_y = lgb_model.predict(test_x)
lightGBM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1000
           1       1.00      0.97      0.98      1000
           2       0.80      0.83      0.81      1000
           3       0.89      0.90      0.90      1000
           4       0.80      0.83      0.82      1000
           5       0.99      0.97      0.98      1000
           6       0.72      0.66      0.69      1000
           7       0.95      0.97      0.96      1000
           8       0.98      0.97      0.98      1000
           9       0.97      0.96      0.96      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [134]:
# dont run this part, i'm still tuning
err_list = []
for i in np.arange(0.0, 1.0, 0.1):
    model = LGBMClassifier(objective='multiclass',path_smooth = i)
    model.fit(X_train,y_train,categorical_feature=[0,3])
    predictions = model.predict(X_test)
    error = sum(predictions!=y_test)/len(y_test)
    err_list.append(error)
err_list

C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\asus\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Us

[0.0939, 0.0947, 0.0945, 0.0936, 0.0943, 0.0927, 0.0952, 0.094, 0.0941, 0.0948]

# LDA

In [135]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [138]:
lda = LDA(n_components=9)

In [139]:
expected_y  = test_labels
lda_model = lda.fit(train_x, train_labels)
predicted_y = lda_model.predict(test_x)
lda_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.81      0.78      0.79      1000
           1       1.00      0.93      0.96      1000
           2       0.70      0.68      0.69      1000
           3       0.80      0.85      0.82      1000
           4       0.70      0.74      0.72      1000
           5       0.89      0.89      0.89      1000
           6       0.54      0.56      0.55      1000
           7       0.88      0.89      0.89      1000
           8       0.94      0.92      0.93      1000
           9       0.91      0.91      0.91      1000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



In [140]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.808533,0.7770,0.792453,1000.0000
1,0.996795,0.9330,0.963843,1000.0000
2,0.703093,0.6820,0.692386,1000.0000
3,0.801136,0.8460,0.822957,1000.0000
4,0.700943,0.7430,0.721359,1000.0000
5,0.886454,0.8900,0.888224,1000.0000
6,0.544922,0.5580,0.551383,1000.0000
7,0.883284,0.8930,0.888115,1000.0000
8,0.935910,0.9200,0.927887,1000.0000
9,0.913568,0.9090,0.911278,1000.0000


# SVM

In [141]:
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [98]:
# Use Ploynomial kernel 
svmpoly = svm.SVC(kernel='poly', degree=3, C=1).fit(train_x, train_labels)

In [113]:
expected_y  = test_labels
predicted_y = svmpoly.predict(test_x)
SVM_pred_y = predicted_y
print(metrics.classification_report(expected_y, predicted_y))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81      1000
           1       0.99      0.95      0.97      1000
           2       0.80      0.78      0.79      1000
           3       0.88      0.86      0.87      1000
           4       0.83      0.76      0.79      1000
           5       0.84      0.96      0.89      1000
           6       0.66      0.69      0.68      1000
           7       0.94      0.91      0.92      1000
           8       0.97      0.94      0.95      1000
           9       0.96      0.94      0.95      1000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000



In [101]:
report = metrics.classification_report(expected_y, predicted_y, output_dict=True)
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.790019,0.839,0.813773,1000.000
1,0.991693,0.955,0.973001,1000.000
2,0.804145,0.776,0.789822,1000.000
3,0.878695,0.862,0.870268,1000.000
4,0.825708,0.758,0.790407,1000.000
5,0.835366,0.959,0.892924,1000.000
6,0.662500,0.689,0.675490,1000.000
7,0.936410,0.913,0.924557,1000.000
8,0.966084,0.940,0.952864,1000.000
9,0.963077,0.939,0.950886,1000.000


In [ ]:
# to get the precision dictionary
out = {}
for i in range(0,10):
    out.update({i:round(report[str(i)]['precision'],2)})

In [128]:
# create df of y_pred of test set for each model
y_pred_test = pd.concat([pd.DataFrame(lightGBM_pred_y, columns = ['LightGBM']),
           pd.DataFrame(lda_pred_y, columns = ['LDA']), 
           pd.DataFrame(SVM_pred_y, columns = ['SVM'])],axis=1)

In [130]:
y_pred_test.to_csv('y_pred_test.csv')